In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of currencies
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    currencies = ['currency_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + currencies + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
                 
        else: ### country - currency
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(currencies[pair])        
            
        sentence += list(np.random.choice(randoms, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [10]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_currencies = []
    prob_currencies = []

    for _ in range(n_samples):
        sentence = []
        random_currencies = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_currency in random_currencies:
            sentence.append('country_' + str(random_currency))
            sentence.append('currency_' + str(random_currency))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['currency_' + str(random_currencies[-1])]
        acc_currencies.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_currencies.append(temp[0][vocab_map['currency_' + str(random_currencies[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_currencies), np.mean(prob_currencies))

In [11]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 100 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [12]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 5.796033e-24)
(0.0, 4.7025217e-24)
(0.0, 4.7149376e-24)
(0.0, 4.4209773e-24)
(0.0, 4.438526e-23)
(0.0, 4.3045593e-23)
(0.0, 2.8419175e-22)
(0.0, 2.2080136e-22)
(0.0, 9.940822e-24)
(0.0, 1.04792575e-23)
(0.0, 3.078722e-23)
(0.0, 2.0423134e-23)
(0.0, 6.4996226e-24)
(0.0, 7.9621995e-24)
(0.0, 3.408337e-23)
(0.0, 5.1247184e-23)
(0.0, 1.5359316e-23)
(0.0, 1.454771e-23)
(0.0, 1.2405199e-23)
(0.0, 1.355553e-23)
(0.0, 4.4816352502630916e-23)
(0.0, 3.9118546775485297e-23)


In [13]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 1.83211e-23)
(0.0, 1.0684057e-23)
(0.0, 7.873381e-24)
(0.0, 5.073485e-23)
(0.0, 1.4105732e-28)
(0.0, 1.2614312e-28)
(0.0, 1.0712223e-25)
(0.0, 1.3483972e-25)
(0.0, 2.0385686e-29)
(0.0, 1.09758544e-29)
(0.0, 2.3913352e-24)
(0.0, 2.9132454e-24)
(0.0, 2.1551773e-22)
(0.0, 3.0339206e-21)
(0.0, 2.2185024e-25)
(0.0, 8.237853e-26)
(0.0, 1.3698584e-23)
(0.0, 1.7828996e-23)
(0.0, 9.806989e-27)
(0.0, 2.709645e-26)
(0.0, 2.5814107458837636e-23)
(0.0, 3.1163261714555263e-22)


In [14]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.292, 0.43100488)
(0.718, 0.48154968)
(0.606, 0.4597223)
(0.423, 0.43346715)
(0.489, 0.46049324)
(0.482, 0.4635778)
(0.605, 0.45658067)
(0.441, 0.43434736)
(0.577, 0.4507912)
(0.4, 0.43029433)
(0.517, 0.4483996)
(0.494, 0.44827104)
(0.404, 0.4360253)
(0.602, 0.4612461)
(0.435, 0.44648066)
(0.579, 0.46808347)
(0.603, 0.45322472)
(0.366, 0.43494546)
(0.547, 0.42392468)
(0.461, 0.40429127)
(0.5075, 0.44666472673416135)
(0.49660000000000004, 0.44600736796855933)


In [15]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(0.9999999999999999, 0.9999999999999999)
(0.9999999999999999, 0.9999999999999999)


In [18]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99821)
(1.0, 0.9985706)
(1.0, 0.99828297)
(1.0, 0.99878734)
(1.0, 0.99817836)
(1.0, 0.99862856)
(1.0, 0.9989093)
(1.0, 0.99880165)
(1.0, 0.9984612)
(1.0, 0.9987806)
(1.0, 0.9986429)
(1.0, 0.99872255)
(1.0, 0.9988089)
(1.0, 0.9986223)
(1.0, 0.99861455)
(1.0, 0.9981515)
(1.0, 0.9990925)
(1.0, 0.9987157)
(1.0, 0.9987347)
(1.0, 0.9989423)
(0.9999999999999999, 0.9985935449600221)
(0.9999999999999999, 0.9986723124980926)


In [17]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9995824)
(1.0, 0.9994664)
(1.0, 0.9995741)
(1.0, 0.9996023)
(1.0, 0.99964833)
(1.0, 0.9996529)
(1.0, 0.9996911)
(1.0, 0.9996059)
(1.0, 0.9996221)
(1.0, 0.9996092)
(1.0, 0.99947727)
(1.0, 0.9996098)
(1.0, 0.9996047)
(1.0, 0.9994614)
(1.0, 0.9996802)
(1.0, 0.99957)
(1.0, 0.9996869)
(1.0, 0.9996523)
(1.0, 0.9994595)
(1.0, 0.99945885)
(0.9999999999999999, 0.9996026694774628)
(0.9999999999999999, 0.9995689153671264)
